In [1]:
ANIMALS = ('Avestruz', 'Águia', 'Burro', 'Borboleta', 'Cachorro', 'Cabra', 'Carneiro', 'Camelo', 'Cobra', 'Coelho', 'Cavalo', 'Elefante', 'Galo', 'Gato', 'Jacaré', 'Leão', 'Macaco', 'Porco', 'Pavão', 'Peru', 'Touro', 'Tigre', 'Urso', 'Veado', 'Vaca',)

In [2]:
def animal_id_from_digits(digits):
    i = (int(digits[-2:])-1)//4
    if i < 0:
        i = 24
    return str(i).zfill(2)

In [3]:
from collections import Counter

class Round:

    def __init__(self, choices, draw, animal=False):
        self.choices = choices
        if animal:
            self.draw = [animal_id_from_digits(d) for d in draw]
        else:
            self.draw = [d[-len(choices[0]):] for d in draw]

    def check_present(self):
        prev_count = sum(Counter(self.draw).values())
        after_removal = sum( (Counter(self.draw) - Counter(self.choices)).values() )
        return prev_count - after_removal == len(self.choices)
    
    def check_ordered(self):
        length = len(self.choices)
        try:
            first_idx = self.draw.index(self.choices[0])
        except ValueError:
            return False
        return self.choices == self.draw[first_idx:first_idx+length]
    
    def check_exact(self):
        matches = [self.choices[i] == self.draw[i] for i in range(len(self.choices))]
        return all(matches)

In [4]:
import numpy as np

class GameMeta(type):

    __type_instances = {}

    def __new__(cls, name, bases, clsdict):
        
        clsobj = super().__new__(cls, name, bases, clsdict)
        if not hasattr(clsobj, 'prize'):
            setattr(clsobj, 'prize', 2)
        if not hasattr(clsobj, 'match_index'):
            setattr(clsobj, 'match_index', False)
        if not hasattr(clsobj, 'match_order'):
            setattr(clsobj, 'match_order', False)
        if not hasattr(clsobj, 'n_choices'):
            setattr(clsobj, 'n_choices', 1)
        if not hasattr(clsobj, 'len_choice'):
            setattr(clsobj, 'len_choice', 2)
        if not hasattr(clsobj, 'animal'):
            setattr(clsobj, 'animal', False)
        if not hasattr(clsobj, 'desc'):
            setattr(clsobj, 'desc', 'missing desc')
        if clsobj.animal and clsobj.len_choice != 2:
            raise TypeError('Conflicting settings: animal=True implies len_choice=2')
        if clsobj.match_index and clsobj.match_order:
            print(f'Warning {name} class: match_index=True takes precedence over match_order')

        # Start inner functions:

        def check_win(self, choices, draw):
            if clsobj.n_choices != len(choices):
                raise TypeError(f'Expected {clsobj.n_choices} choices, got {len(choices)}')
            if not all([len(c) == clsobj.len_choice for c in choices]):
                raise TypeError(f'Choices must be all of length {clsobj.len_choice}')
            _draw =[d[-(clsobj.len_choice):] for d in draw]
            game_round = Round(choices=choices, draw=draw, animal=clsobj.animal)
            if clsobj.match_index:
                return game_round.check_exact()
            elif clsobj.match_order:
                return game_round.check_ordered()
            else:
                return game_round.check_present()

        def calc(self, bet, choices, draw):
            if self.check_win(choices, draw):
                return (bet * clsobj.prize) - bet
            else:
                return -bet

        def random(self):
            high = 24 if clsobj.animal else 10**clsobj.len_choice-1
            r = np.random.randint(low=0, high=high, size=clsobj.n_choices)
            return [str(v).zfill(clsobj.len_choice) for v in r]

        # End inner functions.

        setattr(clsobj, 'random', random)
        setattr(clsobj, 'check_win', check_win)
        setattr(clsobj, 'calc', calc)
        setattr(clsobj, 'name', name)
        GameMeta.__type_instances[name] = clsobj()
        return clsobj
    
    @classmethod
    def get_type(cls, name):
        return cls.__type_instances.get(name)
    
    @classmethod
    def get_types(cls):
        return tuple(cls.__type_instances.values())

In [5]:
# DEFAULTS:
#     desc = 'missing desc'
#     animal = False
#     match_order = False
#     match_index = False
#     n_choices = 1
#     len_choice = 2
#     prize = 2

class DQ(metaclass=GameMeta):
    desc = 'Dezena Qualquer'
    prize = 12.0

class DS(metaclass=GameMeta):
    desc = 'Dezena Seca'
    prize = 60.0
    match_index = True

class GQ(metaclass=GameMeta):
    desc = 'Grupo Qualquer'
    animal = True
    prize = 3.6

class GS(metaclass=GameMeta):
    desc = 'Grupo Seco'
    animal = True
    match_index = True
    prize = 18.0

class GD(metaclass=GameMeta):
    desc = 'Grupo Duque'
    animal = True
    prize = 18.5
    n_choices = 2

class GT(metaclass=GameMeta):
    desc = 'Grupo Terno'
    animal = True
    n_choices = 3
    prize = 130.0

class DD(metaclass=GameMeta):
    desc = 'Duque Dezenas'
    prize = 300.0
    n_choices = 2

class DT(metaclass=GameMeta):
    desc = 'Dezena Terno'
    n_choices = 3
    prize = 3000.0

class CQ(metaclass=GameMeta):
    desc = 'Centena Qualquer'
    len_choice = 3
    prize = 120.0

class CS(metaclass=GameMeta):
    desc = 'Centena Seca'
    len_choice = 3
    match_index = True
    prize = 600.0

class MQ(metaclass=GameMeta):
    desc = 'Milhar Qualquer'
    len_choice = 4
    prize = 800.0

class MS(metaclass=GameMeta):
    desc = 'Milhar Seco'
    len_choice = 4
    match_index = True
    prize = 4000.0

# Example:
#
#class TI(metaclass=GameMeta):
#    desc = 'Terno Improvável'
#    animal = True
#    n_choices = 3
#    match_order = True
#    prize = 1200.0

In [6]:
def bet_sample(mean, std, size):
    a = np.random.normal(loc=mean, scale=std, size=size)
    sample = np.around(a, decimals=1)
    return sample[sample > 0]

In [7]:
def random_draw():
    return [str(n).zfill(4) for n in np.random.randint(low=1000, high=9999, size=5)]

In [8]:
import pandas as pd

def make_simulation(bet_mean, bet_std, size, house_pov=False, draw=None):
    df = pd.DataFrame()
    game_types = GameMeta.get_types()
    df['aposta'] = bet_sample(mean=bet_mean, std=bet_std, size=size)
    if draw is None:
        draw = random_draw()
    jogos = []
    tipos = []
    siglas = []
    saldo = []
    for bet in df['aposta']:
        game = np.random.choice(game_types)
        tipos.append(game.desc)
        siglas.append(game.name)
        j = game.random()
        s = game.calc(bet=bet, draw=draw, choices=j)
        if house_pov:
            s *= -1
        if game.animal:
            j = [ANIMALS[int(i)] for i in j]
        jogos.append(j)
        saldo.append(s)
    df['tipo_jogo'] = tipos
    df['sigla'] = siglas
    df['jogo'] = jogos
    df['saldo'] = saldo
    df['sorteio'] = [draw for _ in range(len(df))]
    return df

In [9]:
import locale
locale.setlocale( locale.LC_ALL, '')

'LC_CTYPE=en_US.UTF-8;LC_NUMERIC=pt_BR.UTF-8;LC_TIME=en_US.UTF-8;LC_COLLATE=en_US.UTF-8;LC_MONETARY=pt_BR.UTF-8;LC_MESSAGES=en_US.UTF-8;LC_PAPER=pt_BR.UTF-8;LC_NAME=pt_BR.UTF-8;LC_ADDRESS=pt_BR.UTF-8;LC_TELEPHONE=pt_BR.UTF-8;LC_MEASUREMENT=pt_BR.UTF-8;LC_IDENTIFICATION=pt_BR.UTF-8'

In [10]:
def regroup_by_type(source):
    df = pd.DataFrame()
    saldos = source[['sigla', 'saldo']].groupby(['sigla']).sum()
    df['sigla'] = saldos.index
    df['tipo_jogo'] = [GameMeta.get_type(s).desc for s in df['sigla']]
    df['n'] = source.groupby(['sigla']).count().saldo.values
    wins = source[source['saldo'] < 0].groupby(['sigla']).count().jogo
    df['acertos'] = [wins.get(k, 0) for k in df['sigla'].values]
    probs = [np.around(a / b, decimals=5) * 100 for a, b in zip(df['acertos'].values, df['n'].values)]
    df['p'] = [f'{prob:.3f}%' for prob in probs]
    df['saldo'] = [s for s in saldos.saldo]
    df.sort_values('saldo', inplace=True)
    avg = np.around(df['acertos'].sum()/df['n'].sum(), decimals=5)*100
    df.loc[len(df)] = ['', 'TOTAL', f"{df['n'].sum():_}", f"{df['acertos'].sum():_}", f'{avg:.3f}%', df['saldo'].sum()]
    df['saldo'] = [locale.currency(s, grouping=True) for s in df['saldo'].values]
    df['n'] = [f'{int(a):_}' for a in df['n']]
    df['acertos'] = [f'{int(a):_}' for a in df['acertos']]
    df.index = list(range(len(df)))
    return df

In [11]:
from contextlib import contextmanager
import time

@contextmanager
def timer():
    before = time.time()
    yield
    after = time.time()
    print(f'Block executed in {after - before:.2f} seconds')

In [12]:
lottery_draw = random_draw()
animal_names = [ANIMALS[int(animal_id_from_digits(n))] for n in lottery_draw]
print(animal_names)
lottery_draw

['Carneiro', 'Jacaré', 'Peru', 'Galo', 'Galo']


['7225', '9960', '5879', '7352', '1249']

In [13]:
with timer():
    sim = make_simulation(draw=lottery_draw, bet_mean=11, bet_std=5, size=200_200, house_pov=True).sort_values('saldo')

Block executed in 9.92 seconds


In [14]:
with timer():
    sim_display = sim.copy()
    sim_display['saldo'] = [locale.currency(s, grouping=True) for s in sim_display['saldo'].values]
sim_display.head(10)

Block executed in 3.02 seconds


,aposta,tipo_jogo,sigla,jogo,saldo,sorteio
8684,12.3,Dezena Terno,DT,"[60, 25, 49]","-R$ 36.887,70","[7225, 9960, 5879, 7352, 1249]"
192531,8.6,Milhar Seco,MS,[7225],"-R$ 34.391,40","[7225, 9960, 5879, 7352, 1249]"
185749,8.6,Dezena Terno,DT,"[25, 52, 79]","-R$ 25.791,40","[7225, 9960, 5879, 7352, 1249]"
60,20.6,Milhar Qualquer,MQ,[7225],"-R$ 16.459,40","[7225, 9960, 5879, 7352, 1249]"
27039,17.5,Milhar Qualquer,MQ,[7352],"-R$ 13.982,50","[7225, 9960, 5879, 7352, 1249]"
19306,21.6,Centena Seca,CS,[225],"-R$ 12.938,40","[7225, 9960, 5879, 7352, 1249]"
22023,15.3,Milhar Qualquer,MQ,[1249],"-R$ 12.224,70","[7225, 9960, 5879, 7352, 1249]"
166887,19.6,Centena Seca,CS,[225],"-R$ 11.740,40","[7225, 9960, 5879, 7352, 1249]"
115745,14.6,Milhar Qualquer,MQ,[9960],"-R$ 11.665,40","[7225, 9960, 5879, 7352, 1249]"
28451,14.4,Milhar Qualquer,MQ,[1249],"-R$ 11.505,60","[7225, 9960, 5879, 7352, 1249]"


In [15]:
with timer():
    by_type = regroup_by_type(source=sim)
by_type

Block executed in 0.15 seconds


,sigla,tipo_jogo,n,acertos,p,saldo
0,GS,Grupo Seco,16_616,723,4.351%,"R$ 43.069,00"
1,CQ,Centena Qualquer,16_193,84,0.519%,"R$ 63.079,30"
2,DS,Dezena Seca,16_405,169,1.030%,"R$ 72.332,60"
3,GQ,Grupo Qualquer,16_560,2_795,16.878%,"R$ 73.040,24"
4,DQ,Dezena Qualquer,16_408,838,5.107%,"R$ 73.928,80"
5,CS,Centena Seca,16_456,16,0.097%,"R$ 77.979,40"
6,DD,Duque Dezenas,16_455,27,0.164%,"R$ 92.153,80"
7,MQ,Milhar Qualquer,16_395,7,0.043%,"R$ 105.347,70"
8,GD,Grupo Duque,16_574,371,2.238%,"R$ 106.886,20"
9,DT,Dezena Terno,16_456,2,0.012%,"R$ 120.524,60"


In [16]:
last_type = GameMeta.get_types()[-1]
mask = sim['sigla'] == last_type.name
games = sim[mask].copy()
games['saldo'] = [locale.currency(s, grouping=True) for s in games['saldo'].values]
games.head(8)

,aposta,tipo_jogo,sigla,jogo,saldo,sorteio
192531,8.6,Milhar Seco,MS,[7225],"-R$ 34.391,40","[7225, 9960, 5879, 7352, 1249]"
50008,0.1,Milhar Seco,MS,[8595],"R$ 0,10","[7225, 9960, 5879, 7352, 1249]"
120354,0.1,Milhar Seco,MS,[7891],"R$ 0,10","[7225, 9960, 5879, 7352, 1249]"
108112,0.1,Milhar Seco,MS,[9750],"R$ 0,10","[7225, 9960, 5879, 7352, 1249]"
149763,0.1,Milhar Seco,MS,[3379],"R$ 0,10","[7225, 9960, 5879, 7352, 1249]"
107912,0.1,Milhar Seco,MS,[6222],"R$ 0,10","[7225, 9960, 5879, 7352, 1249]"
33563,0.1,Milhar Seco,MS,[5027],"R$ 0,10","[7225, 9960, 5879, 7352, 1249]"
29712,0.1,Milhar Seco,MS,[2627],"R$ 0,10","[7225, 9960, 5879, 7352, 1249]"


In [17]:
mask = sim['sigla'] == 'GT'
gt = sim[mask].copy()
gt['saldo'] = [locale.currency(s, grouping=True) for s in gt['saldo'].values]
gt.head(12)

,aposta,tipo_jogo,sigla,jogo,saldo,sorteio
94204,23.8,Grupo Terno,GT,"[Galo, Carneiro, Galo]","-R$ 3.070,20","[7225, 9960, 5879, 7352, 1249]"
66171,23.2,Grupo Terno,GT,"[Carneiro, Jacaré, Peru]","-R$ 2.992,80","[7225, 9960, 5879, 7352, 1249]"
12040,22.5,Grupo Terno,GT,"[Carneiro, Peru, Galo]","-R$ 2.902,50","[7225, 9960, 5879, 7352, 1249]"
156959,20.3,Grupo Terno,GT,"[Jacaré, Peru, Galo]","-R$ 2.618,70","[7225, 9960, 5879, 7352, 1249]"
12508,17.3,Grupo Terno,GT,"[Galo, Jacaré, Peru]","-R$ 2.231,70","[7225, 9960, 5879, 7352, 1249]"
185021,15.9,Grupo Terno,GT,"[Galo, Peru, Galo]","-R$ 2.051,10","[7225, 9960, 5879, 7352, 1249]"
70488,15.6,Grupo Terno,GT,"[Peru, Galo, Jacaré]","-R$ 2.012,40","[7225, 9960, 5879, 7352, 1249]"
139900,14.6,Grupo Terno,GT,"[Jacaré, Galo, Peru]","-R$ 1.883,40","[7225, 9960, 5879, 7352, 1249]"
114056,14.3,Grupo Terno,GT,"[Jacaré, Galo, Peru]","-R$ 1.844,70","[7225, 9960, 5879, 7352, 1249]"
28002,13.1,Grupo Terno,GT,"[Galo, Galo, Jacaré]","-R$ 1.689,90","[7225, 9960, 5879, 7352, 1249]"


In [18]:
animal_names

['Carneiro', 'Jacaré', 'Peru', 'Galo', 'Galo']